In [0]:
!git clone https://github.com/google-research/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 300.28 KiB | 4.00 MiB/s, done.
Resolving deltas: 100% (185/185), done.


In [0]:
ls -l

total 12
drwxr-xr-x 3 root root 4096 Mar  9 05:16 bert/
drwx------ 3 root root 4096 Mar  9 05:16 drive/
drwxr-xr-x 1 root root 4096 Mar  3 18:11 sample_data/


In [0]:
cd bert

/content/bert


In [0]:
ls -l

total 396
-rw-r--r-- 1 root root  1323 Mar  9 05:16 CONTRIBUTING.md
-rw-r--r-- 1 root root 16475 Mar  9 05:16 create_pretraining_data.py
-rw-r--r-- 1 root root 13898 Mar  9 05:16 extract_features.py
-rw-r--r-- 1 root root   616 Mar  9 05:16 __init__.py
-rw-r--r-- 1 root root 11358 Mar  9 05:16 LICENSE
-rw-r--r-- 1 root root 37922 Mar  9 05:16 modeling.py
-rw-r--r-- 1 root root  9191 Mar  9 05:16 modeling_test.py
-rw-r--r-- 1 root root 11242 Mar  9 05:16 multilingual.md
-rw-r--r-- 1 root root  6258 Mar  9 05:16 optimization.py
-rw-r--r-- 1 root root  1721 Mar  9 05:16 optimization_test.py
-rw-r--r-- 1 root root 66488 Mar  9 05:16 predicting_movie_reviews_with_bert_on_tf_hub.ipynb
-rw-r--r-- 1 root root 45390 Mar  9 05:16 README.md
-rw-r--r-- 1 root root   110 Mar  9 05:16 requirements.txt
-rw-r--r-- 1 root root 34783 Mar  9 05:16 run_classifier.py
-rw-r--r-- 1 root root 11426 Mar  9 05:16 run_classifier_with_tfhub.py
-rw-r--r-- 1 root root 18667 Mar  9 05:16 run_pretraining.py
-rw-r--r-

In [0]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip

--2020-03-09 05:17:18--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 2607:f8b0:4001:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1247797031 (1.2G) [application/zip]
Saving to: ‘uncased_L-24_H-1024_A-16.zip’

uncased_L-24_H-1024 100%[===================>]   1.16G   131MB/s    in 8.3s    

2020-03-09 05:17:26 (143 MB/s) - ‘uncased_L-24_H-1024_A-16.zip’ saved [1247797031/1247797031]



In [0]:
!unzip uncased_L-24_H-1024_A-16.zip

Archive:  uncased_L-24_H-1024_A-16.zip
   creating: uncased_L-24_H-1024_A-16/
  inflating: uncased_L-24_H-1024_A-16/bert_model.ckpt.meta  
  inflating: uncased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-24_H-1024_A-16/vocab.txt  
  inflating: uncased_L-24_H-1024_A-16/bert_model.ckpt.index  
  inflating: uncased_L-24_H-1024_A-16/bert_config.json  


In [0]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2020-03-09 05:18:12--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.108.153, 185.199.111.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M  59.2MB/s    in 0.7s    

2020-03-09 05:18:13 (59.2 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2020-03-09 05:18:15--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.110.153, 185.199.111.153, 185.199.109.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       

In [0]:
%tensorflow_version 1.x

In [0]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
print(tf.__version__)

1.15.0


In [0]:
assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is => ', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is =>  grpc://10.0.30.2:8470
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 13684399384723840451),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 17528208054503732205),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 17647100610726072989),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 1783542691872131306),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 10910674333768463561),
 _DeviceAttributes(/job:tpu_worker/replica:0/task

In [0]:
BUCKET = 'bertnlpusecase' #@param {type:"string"}
assert BUCKET, '*** Must specify an existing GCS bucket name ***'
output_dir_name = 'bert_output' #@param {type:"string"}
BUCKET_NAME = 'gs://{}'.format(BUCKET)
OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET,output_dir_name)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: gs://bertnlpusecase/bert_output *****


In [0]:
!gsutil mv /content/bert/uncased_L-24_H-1024_A-16 $BUCKET_NAME

Copying file:///content/bert/uncased_L-24_H-1024_A-16/bert_model.ckpt.index [Content-Type=application/octet-stream]...
Removing file:///content/bert/uncased_L-24_H-1024_A-16/bert_model.ckpt.index...
Copying file:///content/bert/uncased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite ob

In [0]:
!python run_squad.py \
  --vocab_file=$BUCKET_NAME/uncased_L-24_H-1024_A-16/vocab.txt \
  --bert_config_file=$BUCKET_NAME/uncased_L-24_H-1024_A-16/bert_config.json \
  --init_checkpoint=$BUCKET_NAME/uncased_L-24_H-1024_A-16/bert_model.ckpt \
  --do_train=True \
  --train_file=train-v2.0.json \
  --do_predict=True \
  --predict_file=dev-v2.0.json \
  --train_batch_size=24 \
  --learning_rate=3e-5 \
  --num_train_epochs=2.0 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --version_2_with_negative=True \
  --output_dir=$OUTPUT_DIR  \
  --use_tpu=True \
  --tpu_name=$TPU_ADDRESS

Streaming output truncated to the last 5000 lines.
I0309 06:56:09.052364 140092419307392 tpu_estimator.py:604] Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
I0309 06:56:09.070882 140092419307392 tpu_estimator.py:600] Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
I0309 06:56:09.071180 140092419307392 tpu_estimator.py:604] Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
I0309 06:56:09.089285 140092419307392 tpu_estimator.py:600] Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
I0309 06:56:09.089519 140092419307392 tpu_estimator.py:604] Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
I0309 06:56:09.110263 140092419307392 tpu_estimator.py:600] Enqueue next (1) batch(es) of data

In [0]:
!touch input_file.json

In [0]:
%%writefile input_file.json
{
    "version": "v2.0",
    "data": [
        {
            "title": "your_title",
            "paragraphs": [
                {
                    "qas": [
                        {
                            "question": "When will be next recession?",
                            "id": "56ddde6b9a695914005b9628",
                            "is_impossible": ""
                        },
                        {
                            "question": "Is there a pandemic?",
                            "id": "56ddde6b9a695914005b9629",
                            "is_impossible": ""
                        },
                        {
                            "question": "When will prices drop?",
                            "id": "56ddde6b9a695914005b962a",
                            "is_impossible": ""
                        }
                    ],
                    "context": "Financial market is not doing good these days. There is a global slowdown due to Corona virus. Commodity prices are rising."                
                 }
            ]
        }
    ]
}

Overwriting input_file.json


In [0]:
OUT_PREDICT = 'gs://{}/{}'.format(BUCKET,'predict_output')
tf.gfile.MakeDirs(OUT_PREDICT)
print('***** Prediction output directory: {} *****'.format(OUT_PREDICT))

***** Prediction output directory: gs://bertnlpusecase/predict_output *****


In [0]:
OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET,output_dir_name)
print('***** Prediction output directory: {} *****'.format(OUTPUT_DIR))

***** Prediction output directory: gs://bertnlpusecase/bert_output *****


In [0]:
!python run_squad.py \
  --vocab_file=$BUCKET_NAME/uncased_L-24_H-1024_A-16/vocab.txt \
  --bert_config_file=$BUCKET_NAME/uncased_L-24_H-1024_A-16/bert_config.json \
  --init_checkpoint=$OUTPUT_DIR/model.ckpt-10859 \
  --do_train=False \
  --max_query_length=30  \
  --do_predict=True \
  --predict_file=input_file.json \
  --predict_batch_size=8 \
  --n_best_size=3 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --output_dir=$OUT_PREDICT/




W0309 10:20:50.773493 139695195903872 module_wrapper.py:139] From run_squad.py:1127: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0309 10:20:50.773682 139695195903872 module_wrapper.py:139] From run_squad.py:1127: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0309 10:20:50.773820 139695195903872 module_wrapper.py:139] From /content/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0309 10:20:53.061390 139695195903872 module_wrapper.py:139] From run_squad.py:1133: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related op